In [ ]:
#Import Relevent Packages 
import datetime
import os
import requests
import time
     
!pip install --quiet geopandas
import pandas as pd
import geopandas as gpd

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [2]:
#Import Relevent Packages 
%%capture
if 'google.colab' in str(get_ipython()):
    !pip install --quiet rioxarray

#Install mapping packages
import rioxarray as rxr
from rioxarray.merge import merge_arrays
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [3]:
#Prep for API Call
# Set root URL for API requests
root_url = 'https://api.climateengine.org/'
# Authentication info for the API (INSERT YOUR OWN KEY)
headers = {'Authorization': 'INSERT YOUR API KEY HERE'}
# Google Storage bucket for storing output files (INSERT YOUR OWN BUCKET)
bucket_name = 'INSERT YOUR BUCKET HERE'

In [ ]:
#Bring in spatial file of interest (GeoJSON, SHP) 
df = gpd.read_file('/content/Continental_US.shp')

 #Generate a bounding box around AOI
bbox = df.bounds

#Get values from box
sw_long = round(bbox.iat[0,0], 6)
sw_lat = round(bbox.iat[0,1], 6)
ne_long = round(bbox.iat[0,2], 6)
ne_lat = round(bbox.iat[0,3], 6)

#Generate Bounding Box Coordinates String
allot_bbox = [sw_long,sw_lat,ne_long,ne_lat]

print(allot_bbox)

In [ ]:
#Select endpoint that exports a map of values to a Google Cloud storage bucket
trendEndpoint = 'raster/export/mann_kendall'

# No need to include an extension on the export_filename below, .tif will be appended automatically.
trendExport_filename = 'LST_Trend_Example'

#Set up parameters dictionary for API call
trendParams = {
    'dataset': 'MODIS_TERRA_8DAY',
    'variable': 'LST_Day_1km',
    'temporal_statistic': 'mean',
    'calculation': 'mk_sen',
    'start_season': '04-20',
    'end_season': '09-30',
    'start_year': '2000',
    'end_year': '2022',
    'bounding_box': f'{allot_bbox}',
    'export_path': f'{bucket_name}/{trendExport_filename}'
}

# Send API request
trendR = requests.get(root_url + trendEndpoint, params=trendParams, headers=headers, verify=False)
trendExport_response = trendR.json()
print(trendExport_response)
print(trendR.json)
print(trendR.text)
print(trendR.status_code)

In [ ]:
#Connect to GCS
#Get access to Google Cloud Storage Bucket 
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects (INSERT GOOGLE PROJECT ID)
project_id = 'INSERT YOUR GOOGLE PROJECT ID HERE'
!gcloud config set project {project_id}

In [ ]:
#Bring in Rasters from GCS
# Download the file from a given Google Cloud Storage bucket.
!gsutil cp gs://{bucket_name}/{trendExport_filename}.tif /content/LST_Trend_Example.tif

In [9]:
#Define file path and read in raster
file_path1 = '/content/LST_Trend_Example.tif'
rds = rxr.open_rasterio(file_path1)

In [ ]:
#Make Maps
# Generate Map 
fig, ax = plt.subplots()
fig.set_size_inches(15, 13)
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
im = ax.imshow(rds[0], cmap ='RdBu_r', vmin = -0.1, vmax = 0.1)

#Add Color Bar
fig.colorbar(im, cax = cax, orientation = 'vertical')

#Add Title
ax.set_title('MODIS Terra 8-Day Slope of Trend in Land Surface Temperature (LST) (Apr 20 to Sep 30, Mean, 2000 - 2022)', fontsize =14)

#Add Figure Export Option
plt.savefig('Trend.png', bbox_inches='tight')

#Show Map
plt.show()